# Step 1: Reading the Given Data

In [1]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('C:/Users/Sampada/Downloads/eng_subtitles_database.db') 


In [2]:
# Read the contents of the README.txt file
with open("C:/Users/Sampada/Downloads/README.txt", 'r') as readme_file:
    readme_contents = readme_file.read()
    

In [3]:
# Display the contents of the README.txt file
print(readme_contents)


Database contains a sample of 82498 subtitle files from opensubtitles.org. 

Most of the subtitles are of movies and tv-series which were released after 1990 and before 2024.

Database File Name: eng_subtitles_database.db
Database contains a table called 'zipfiles' with three columns.
1. num: Unique Subtitle ID reference for www.opensubtitles.org 
2. name: Subtitle File Name
3. content: Subtitle file were compressed and stored as a binary using 'latin-1' encoding.



You can use 'num' to get more details about each subtitle by going to the following link:
https://www.opensubtitles.org/en/subtitles/{num}
**Replace {num} with Unique Subtitle ID.


In [4]:
# Close the database connection
conn.close()


# Step 2: Preprocessing the data

In [5]:
import pandas as pd

# Read subtitle documents from the database into a DataFrame
conn = sqlite3.connect('C:/Users/Sampada/Downloads/eng_subtitles_database.db')
se = pd.read_sql_query("SELECT * FROM zipfiles", conn)  


In [6]:
# Performing basic preprocessing steps
# Let's check the shape of the DataFrame
print("Shape of the DataFrame:")
print(se.shape)


Shape of the DataFrame:
(82498, 3)


In [9]:
# Check the column names to identify the correct one
print(se.columns)


Index(['num', 'name', 'content'], dtype='object')


In [7]:
# Let's examine the first few rows of the DataFrame
print("\nFirst few rows of the DataFrame:")
print(se.head())



First few rows of the DataFrame:
       num                                               name  \
0  9180533                         the.message.(1976).eng.1cd   
1  9180583  here.comes.the.grump.s01.e09.joltin.jack.in.bo...   
2  9180592    yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd   
3  9180594    yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd   
4  9180600                              broker.(2022).eng.1cd   

                                             content  
0  b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1c\xa9\x...  
1  b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x17\xb9\x...  
2  b'PK\x03\x04\x14\x00\x00\x00\x08\x00L\xb9\x99V...  
3  b'PK\x03\x04\x14\x00\x00\x00\x08\x00U\xa9\x99V...  
4  b'PK\x03\x04\x14\x00\x00\x00\x08\x001\xa9\x99V...  


In [8]:
# Let's examine the last few rows of the DataFrame
print("\nLast few rows of the DataFrame:")
print(se.tail())



Last few rows of the DataFrame:
           num                                        name  \
82493  9521935            the.prophets.game.(2000).eng.1cd   
82494  9521937                  west.beirut.(1998).eng.1cd   
82495  9521938  frankenstein.the.true.story.(1973).eng.1cd   
82496  9521940  frankenstein.the.true.story.(1973).eng.1cd   
82497  9521941       zombie.island.massacre.(1984).eng.1cd   

                                                 content  
82493  b'PK\x03\x04\x14\x00\x00\x00\x08\x00\xb8\xa6\x...  
82494  b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x13\x97\x...  
82495  b'PK\x03\x04\x14\x00\x00\x00\x08\x00$\x97\x9aV...  
82496  b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x00\x97\x...  
82497  b'PK\x03\x04\x14\x00\x00\x00\x08\x00,\x97\x9aV...  


In [10]:
# Close the database connection
conn.close()


# Step - 3: Unzipping & Applying Cleaning Steps on Subtitle Documents¶

In [20]:
import pandas as pd
import sqlite3
import zipfile
import re

# Connect to the SQLite database
conn = sqlite3.connect('C:/Users/Sampada/Downloads/eng_subtitles_database.db')

# Read subtitle documents from the database into a DataFrame
se = pd.read_sql_query("SELECT * FROM zipfiles", conn)  # Replace 'zipfiles' with your table name

# Close the database connection
conn.close()


In [21]:
import io

# Define a function to unzip files and extract text
def unzip_and_extract(binary_data):
    extracted_texts = []
    with io.BytesIO(binary_data) as f:
        with zipfile.ZipFile(f, 'r') as zip_ref:
            for file_name in zip_ref.namelist():
                with zip_ref.open(file_name) as file:
                    extracted_text = file.read().decode('latin-1')  # Decode content from latin-1 encoding
                    cleaned_text = clean_text(extracted_text)
                    extracted_texts.append(cleaned_text)
    return extracted_texts


In [23]:
import re

# Define a function to clean the text
def clean_text(text):
    # Remove timestamps (e.g., [00:00:00])
    cleaned_text = re.sub(r'\[\d+:\d+:\d+\]', '', text)
    # Remove any other special characters or symbols
    cleaned_text = re.sub(r'[^\w\s]', '', cleaned_text)
    # Convert to lowercase
    cleaned_text = cleaned_text.lower()
    # Remove extra whitespaces
    cleaned_text = ' '.join(cleaned_text.split())
    return cleaned_text


In [26]:
# Apply the clean_text function to each extracted text
se['cleaned_text'] = se['content'].apply(unzip_and_extract).str.join('').apply(clean_text)


In [27]:
# Display the DataFrame with the added 'cleaned_text' column
print(se.head())


       num                                               name  \
0  9180533                         the.message.(1976).eng.1cd   
1  9180583  here.comes.the.grump.s01.e09.joltin.jack.in.bo...   
2  9180592    yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd   
3  9180594    yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd   
4  9180600                              broker.(2022).eng.1cd   

                                             content  \
0  b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1c\xa9\x...   
1  b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x17\xb9\x...   
2  b'PK\x03\x04\x14\x00\x00\x00\x08\x00L\xb9\x99V...   
3  b'PK\x03\x04\x14\x00\x00\x00\x08\x00U\xa9\x99V...   
4  b'PK\x03\x04\x14\x00\x00\x00\x08\x001\xa9\x99V...   

                                        cleaned_text  
0  1 000006000 000012074 watch any video online w...  
1  1 000029359 000032048 ah theres princess dawn ...  
2  1 000053200 000056030 iyumis cells 2i 2 000059...  
3  1 000006000 000012074 watch any video online w...

In [28]:
# Close the database connection
conn.close()


# Step 4: Saving Cleaned Data to CSV Format

In [34]:
pip install pandas


In [39]:
import pandas as pd

In [35]:
# Save the cleaned data to a CSV file
csv_file_path = 'cleaned_data.csv'
se.to_csv(csv_file_path, index=False)

# Print message confirming the saving of the CSV file
print(f"Cleaned data saved to '{csv_file_path}'.")


Cleaned data saved to 'cleaned_data.csv'.


In [38]:
# Load the cleaned data from the CSV file
csv_file_path = 'cleaned_data.csv'
cleaned_data = pd.read_csv(csv_file_path)


In [40]:
# Print basic information about the DataFrame
# Print the shape of dataframe
print("Shape of the DataFrame:")
print(cleaned_data.shape)


Shape of the DataFrame:
(82498, 4)


In [41]:
# Print the column names of dataframe
print("\nColumn names:")
print(cleaned_data.columns)



Column names:
Index(['num', 'name', 'content', 'cleaned_text'], dtype='object')


In [42]:
# Print the first few rows of dataframe
print("\nFirst few rows of the DataFrame:")
print(cleaned_data.head())



First few rows of the DataFrame:
       num                                               name  \
0  9180533                         the.message.(1976).eng.1cd   
1  9180583  here.comes.the.grump.s01.e09.joltin.jack.in.bo...   
2  9180592    yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd   
3  9180594    yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd   
4  9180600                              broker.(2022).eng.1cd   

                                             content  \
0  b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1c\xa9\x...   
1  b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x17\xb9\x...   
2  b'PK\x03\x04\x14\x00\x00\x00\x08\x00L\xb9\x99V...   
3  b'PK\x03\x04\x14\x00\x00\x00\x08\x00U\xa9\x99V...   
4  b'PK\x03\x04\x14\x00\x00\x00\x08\x001\xa9\x99V...   

                                        cleaned_text  
0  1 000006000 000012074 watch any video online w...  
1  1 000029359 000032048 ah theres princess dawn ...  
2  1 000053200 000056030 iyumis cells 2i 2 000059...  
3  1 000006000 000

In [43]:
# Print the last few rows of dataframe
print("\nLast few rows of the DataFrame:")
print(cleaned_data.tail())



Last few rows of the DataFrame:
           num                                        name  \
82493  9521935            the.prophets.game.(2000).eng.1cd   
82494  9521937                  west.beirut.(1998).eng.1cd   
82495  9521938  frankenstein.the.true.story.(1973).eng.1cd   
82496  9521940  frankenstein.the.true.story.(1973).eng.1cd   
82497  9521941       zombie.island.massacre.(1984).eng.1cd   

                                                 content  \
82493  b'PK\x03\x04\x14\x00\x00\x00\x08\x00\xb8\xa6\x...   
82494  b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x13\x97\x...   
82495  b'PK\x03\x04\x14\x00\x00\x00\x08\x00$\x97\x9aV...   
82496  b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x00\x97\x...   
82497  b'PK\x03\x04\x14\x00\x00\x00\x08\x00,\x97\x9aV...   

                                            cleaned_text  
82493  ï1 000116284 000119537 god why are you punishi...  
82494  1 000006000 000012074 apiopensubtitlesorg is d...  
82495  1 000001001 000004630 dramatic orchestral musi...

In [45]:
# Define a function to remove numbers using regular expressions
def remove_numbers(text):
    return re.sub(r'\d+', '', text)


In [46]:
# Apply the function to remove numbers from the 'cleaned_text' column
cleaned_data['cleaned_text'] = cleaned_data['cleaned_text'].apply(remove_numbers)


In [47]:
# Print first few rows of the cleaned data
print(cleaned_data.head())


       num                                               name  \
0  9180533                         the.message.(1976).eng.1cd   
1  9180583  here.comes.the.grump.s01.e09.joltin.jack.in.bo...   
2  9180592    yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd   
3  9180594    yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd   
4  9180600                              broker.(2022).eng.1cd   

                                             content  \
0  b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1c\xa9\x...   
1  b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x17\xb9\x...   
2  b'PK\x03\x04\x14\x00\x00\x00\x08\x00L\xb9\x99V...   
3  b'PK\x03\x04\x14\x00\x00\x00\x08\x00U\xa9\x99V...   
4  b'PK\x03\x04\x14\x00\x00\x00\x08\x001\xa9\x99V...   

                                        cleaned_text  
0     watch any video online with opensubtitles f...  
1     ah theres princess dawn and terry with the ...  
2     iyumis cells i    iepisode  extremely polit...  
3     watch any video online with opensubtitles f...